# Stability of Householder Triangularization

## An experiment

In [25]:
# Create a random upper-triangular matrix
R = triu(rand(50,50));

# Construct a random orthogonal matrix
Q,X = qr(rand(50,50));

# Build a matrix with known QR factorization
A = Q*R;

# Compute QR factorization with Householder Triangularization
Q2,R2 = qr(A);

# Compare errors
norm(Q2-Q), norm(R2-R)/norm(R), norm(A-Q2*R2)/norm(A)

(0.004938420359901829,9.94724397121888e-5,7.015158696105438e-16)

Notice how large the first two errors are compared with the last term!  Somehow even though $Q_2$ and $R_2$ are terrible approximations of $Q$ and $R$, their product $Q_2R_2$ is a surprising approximation of $QR$.  Somehow, the errors introduced in $Q_2$ and $R_2$ are highly correlated and cancel out in the product.

To illustrate how special this cancellation of errors is, we construct another pair of matrices which are small perturbations of $Q$ and $R$

In [2]:
# Compute two slight perturbations of Q and R
Q3 = Q+1e-6*rand(50,50);
R3 = R+1e-6*rand(50,50);

# Find the error
norm(A-Q3*R3)/norm(A)

6.2158685459599515e-6

This time, the error is massive, even though $Q_3$ and $R_3$ are 'just as bad' an approximation of $Q$ and $R$.

The errors in $Q_2$ and $R_2$ are _forward errors_.  In general, a large forward error can be the result of an ill-conditioned problem or an unstable algorithm.

The error in $Q_2R_2$ is the _backward error_ or _residual_.  The smallness of this error suggests that Householder triangularization is backwards stable.

## Theorem

The result of Householder triangularization will take the form: $$\tilde Q \tilde R = A + \delta A,$$ for some small $\delta A$.

Where $\tilde R$ is the upper-triangular matrix that is constructed by Householder triangularization in floating point arithmetic.

However, $\tilde Q$ will be an _exactly unitary_ matrix.   Recall $Q=Q_1Q_2\cdots Q_n$ is a product of Householder reflectors defined by vectors $\mathbf{v}_k$.  In floating point arithmetic we obtain, instead, a sequence $\tilde{\mathbf{v}}_k$ of computed vectors.  Let $\tilde Q_k$ denote the _exactly unitary_ reflector defined **mathematically** not numerically, by $\tilde{\mathbf{v}}_k$.  Then set $\tilde Q = \tilde Q_1\cdots \tilde Q_n$.

> ** THEOREM. ** Let the QR factorization $A=QR$ of a matrix $A\in\mathbb{C}^{m\times n}$ be computed by Householder triangularization, and let the computed factors $\tilde Q$ and $\tilde R$ be defined as above. Then we have: $$\tilde Q \tilde R = A + \delta A, \quad \frac{\|\delta A \|}{\|A\|} = O(\epsilon_{\text{machine}})$$ for some $\delta A \in \mathbb{C}^{m\times n}$. 

## Analyzing an Algorithm to Solve $A\mathbf{x} = \mathbf{b}$

Generally, QR factorization is not an end in itself.  In particular, it is usually a part of a larger algorithm.

We know that the QR facotrization algorithm via Householder triangularization is backwards compatible, however, is it stable enough for use in other algorithms?

As an example we consider the algorithm of solving a linear system via the QR factorization.

In [1]:
# For a given linear system Ax=b
b = rand(50,1);
A = rand(50,50);

# Factor A into QR
Q,R = qr(A);

# Construct Q'b
y = Q'*b;

# Solve the triangular system by back substitution
x = R\y;

b, Q*R*x

(
[0.487946; 0.323111; … ; 0.93473; 0.203352],

[0.487946; 0.323111; … ; 0.93473; 0.203352])

We prove this algorithm is backwards stable by proving each of the three steps is backwards stable.

The first step is QR factorization, leading to matrices $\tilde R$ and $\tilde Q$.   We already know this is backwards stable.

The second step is the computation $\tilde Q^* \mathbf{b}$.  Notice that we are now working with the computed $\tilde Q$, not $Q$.

When $\tilde Q^* \mathbf{b}$ is computed, rounding errors will be introduced, hence the result will not be exactly $\tilde Q^* \mathbf{b}$.  Instead it will be some $\tilde{\mathbf{y}}$.

One shows (exercise!) that: $$(\tilde Q + \delta Q) \tilde{\mathbf{y}} = \mathbf{b}, \quad \|\delta Q\| = O(\epsilon_{\text{machine}}).$$

That is, applying the Householder reflectors in floating point arithmetic is exactly equivalent to multiplying $\mathbf{b}$ by a slightly perturbed matrix $(\tilde Q + \delta Q)^{-1}$.

The final step of this algorithm is back substitution to compute $\tilde{R}^{-1}\tilde{\mathbf{y}}$.  New rounding errors will be introduced.  The estimate in this case (not obvious!): 

$$(\tilde R + \delta R) \tilde{\mathbf{x}} = \tilde{\mathbf{y}}, \quad \frac{\|\delta R\|}{\|\tilde R\|} = O(\epsilon_{\text{machine}}).$$

In words, this asserts that the floating point result $\tilde x$ is exactly the solution of a slight perturbation of $\tilde R\mathbf{x} =\tilde{\mathbf{y}}.$

> ** THEOREM. ** Solving $A\mathbf{x} = \mathbf{b}$ via QR factorization is backwards stable, satisfying: $$(A+\Delta A) \tilde{\mathbf{x}} = \mathbf{b},\quad \frac{\|\Delta A\|}{\|A\|} = O(\epsilon_{\text{machine}}),$$ for some $\Delta A \in \mathbb{C}^{m\times m}$.

** Proof. ** 

We have:

$$
\mathbf{b} = (\tilde Q+\delta Q) (\tilde R + \delta R) \tilde{\mathbf{x}} = \left[\tilde Q\tilde R + (\delta Q) \tilde R + \tilde Q (\delta R) + (\delta Q)(\delta R)\right] \tilde{\mathbf{x}}.
$$

By the previous theorem:

$$\mathbf{b} = \left[A + \delta A + (\delta Q)\tilde R + \tilde Q (\delta R) + (\delta Q)(\delta R)\right]\tilde{\mathbf{x}}$$

Which has the form:

$$
\mathbf{b} = (A + \Delta A) \tilde{\mathbf{x}},
$$
where $\Delta A$ is the above sum of 4 terms.

We now need to bound the error term $\Delta A$.

Since $\tilde Q \tilde R = A + \delta A$ and $\tilde Q$ is unitary:

$$\frac{\|\tilde R\|}{\|A\|} \leq \|\tilde Q^*\| \frac{\|A+\delta A\|}{\|A\|} = O(1),$$ as $\epsilon_{\text{machine}}\to 0$ (by Theorem).

This implies:

$$
\frac{\|(\delta Q) \tilde R\|}{\|A\|} \leq \|\delta Q\| \frac{\|\tilde R\|}{\|A\|} = O(\epsilon_{\text{machine}}).
$$

Similarly:

$$
\frac{\|\tilde Q (\delta R)\|}{\|A\|} \leq \|\tilde Q\| \frac{\|\delta R\|}{\|\tilde R\|}\frac{\|\tilde R\|}{\|A\|} = O(\epsilon_{\text{machine}}).
$$

Finally,

$$
\frac{\|(\delta Q) (\delta R)\|}{\|A\|} \leq \|\delta Q\| \frac{\|\delta R\|}{\|A\|} = O(\epsilon_{\text{machine}}^2).
$$

Therefore the total perturbation $\Delta A$ satisfies:

$$
\frac{\|\Delta A\|}{\|A\|} \leq \frac{\|\delta A\|}{\|A\|} + \frac{\|(\delta Q)\tilde R\|}{\|A\|}+ \frac{\|\tilde Q (\delta R)\|}{\|A\|}+ \frac{\|(\delta Q )(\delta R)\|}{\|A\|} = O(\epsilon_{\text{machine}}).
$$

Combining this result with our general theory of backwards stable algorithms and conditioning of matrix problems, we deduce:

> ** THEOREM. ** The computed solution $\tilde{\mathbf{x}}$ to the system $A\mathbf{x} = \mathbf{b}$ using the above algorithm satisfies: $$\frac{\|\tilde{\mathbf{x}} - \mathbf{x} \|}{\|\mathbf{x}\|} = O(\kappa(A) \epsilon_{\text{machine}}).$$